<a href="https://colab.research.google.com/github/nicolaiberk/nlpdl_project/blob/main/01_Training_PPRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install sklearn

# code is based on Hauke Licht's CAP Model (https://colab.research.google.com/drive/1n7yHr0-lq-hmsXe2sqxLUhq7_4ejLH9o?usp=sharing#scrollTo=CiaoE0V8XZc6)

In [ ]:
# define model
from transformers import BertForSequenceClassification
checkpoint_model = 'distilbert-base-german-cased'
model = BertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

In [3]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/nlpdl/PPRs_cleaned.csv', engine="python")
df = df.reset_index()
df = df.dropna()
# df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [5]:
# train-test split by date
splitdate = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
train = df[df.date < splitdate]
test = df[df.date >= splitdate]


In [6]:
train_texts = list(train["text"])
train_labels = list(train["label"])

test_texts = list(test["text"])
test_labels = list(test["label"])


# there are probably better ways to do this
ulabels = list(set(train_labels))
train_label_dict = {}

for i in range(len(ulabels)):
  train_label_dict[str(ulabels[i])] = i

train_labels = [train_label_dict[str(l)] for l in train_labels]


# and for test set
ulabels = list(set(test_labels))
test_label_dict = {}

for i in range(len(ulabels)):
  test_label_dict[str(ulabels[i])] = i

test_labels = [test_label_dict[str(l)] for l in test_labels]

In [7]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=1234, stratify=train_labels)

print(f"No. obs    -- test: {len(test_labels)}; train: {len(train_labels)}; validation: {len(val_labels)};")
print(f"No. labels -- test: {len(set(test_labels))}; train: {len(set(train_labels))}; validation: {len(set(val_labels))};")

No. obs    -- test: 7052; train: 16522; validation: 4131;
No. labels -- test: 6; train: 6; validation: 6;


In [8]:
# load the tokenizer 
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-german-cased")

# tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [9]:
import torch
class PPRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PPRDataset(train_encodings, train_labels)
val_dataset = PPRDataset(val_encodings, val_labels)
test_dataset = PPRDataset(test_encodings, test_labels)

In [10]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }

In [11]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=3,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=64,
    # number of warmup steps for learning rate scheduler
    warmup_steps=500,
    # strength of weight decay
    weight_decay=0.01,     
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=250,
)


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [13]:
trainer.train()

Step,Training Loss
250,1.246400
500,0.127100
750,0.047600
1000,0.069600
1250,0.041300
1500,0.032300
1750,0.026100
2000,0.019700
2250,0.020700
2500,0.006600


TrainOutput(global_step=3099, training_loss=0.1336287559328944, metrics={'train_runtime': 1492.3844, 'train_samples_per_second': 2.077, 'total_flos': 1.0263315977367552e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1856389120, 'init_mem_gpu_alloc_delta': 271062528, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -100024320, 'train_mem_gpu_alloc_delta': 805930496, 'train_mem_cpu_peaked_delta': 117538816, 'train_mem_gpu_peaked_delta': 6682461696})

## Validation performance

In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
eval_res = trainer.evaluate()

Trainer is attempting to log a value of "[0.99869452 0.99593496 0.99750872 0.99266862 0.99534161 0.98487395]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.99739244 0.9972863  0.99502982 0.99558824 0.99688958 0.98322148]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[1.         0.99458728 1.         0.98976608 0.99379845 0.98653199]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[ 765  739 1001  684  645  297]" of type <class 'numpy.ndarray'> for key "eval/n" as a scalar. This invocation of Tensorboard's writer.add_sc

In [15]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in train_label_dict.items()]
evaluated["class"] = [k for k,v in train_label_dict.items()]
evaluated["f1"] = eval_res["eval_f1"]
evaluated["precision"] = eval_res["eval_precision"]
evaluated["recall"] = eval_res["eval_recall"]
evaluated["n"] = eval_res["eval_n"]

In [16]:
evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
0,0,SPD,0.998695,0.997392,1.000000,765
2,2,Linke,0.997509,0.995030,1.000000,1001
1,1,Greens,0.995935,0.997286,0.994587,739
4,4,FDP,0.995342,0.996890,0.993798,645
3,3,Union,0.992669,0.995588,0.989766,684
5,5,AfD,0.984874,0.983221,0.986532,297


In [17]:
trainer.save_model(os.path.join("drive", "MyDrive", "nlpdl", "01_PPR_model.bin"))

## Test set performance

In [18]:
eval_res = trainer.predict(test_dataset)

In [19]:
eval_res.metrics

{'test_accuracy': 0.8823028927963699,
 'test_f1': array([0.68652038, 0.992078  , 0.96075353, 0.95841854, 0.82421053,
        0.88088045]),
 'test_loss': 0.6543678641319275,
 'test_mem_cpu_alloc_delta': 0,
 'test_mem_cpu_peaked_delta': 0,
 'test_mem_gpu_alloc_delta': 0,
 'test_mem_gpu_peaked_delta': 2517467648,
 'test_n': array([ 661,  820, 1228,  706, 1090, 2547]),
 'test_precision': array([0.52434158, 0.99147381, 0.92727273, 0.92378449, 0.96666667,
        0.97795879]),
 'test_recall': array([0.99394856, 0.99268293, 0.99674267, 0.99575071, 0.71834862,
        0.8013349 ]),
 'test_runtime': 58.824,
 'test_samples_per_second': 119.883}

In [20]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in test_label_dict.items()]
evaluated["class"] = [k for k,v in test_label_dict.items()]
evaluated["f1"] = eval_res.metrics["test_f1"]
evaluated["precision"] = eval_res.metrics["test_precision"]
evaluated["recall"] = eval_res.metrics["test_recall"]
evaluated["n"] = eval_res.metrics["test_n"]

evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
1,1,Greens,0.992078,0.991474,0.992683,820
2,2,Linke,0.960754,0.927273,0.996743,1228
3,3,Union,0.958419,0.923784,0.995751,706
5,5,AfD,0.880880,0.977959,0.801335,2547
4,4,FDP,0.824211,0.966667,0.718349,1090
0,0,SPD,0.686520,0.524342,0.993949,661


In [21]:
pd.crosstab(eval_res.label_ids, eval_res.predictions.argmax(-1))

col_0,0,1,2,3,4,5
row_0,,,,,,
0,657,0,3,0,1,0
1,0,814,2,4,0,0
2,0,0,1224,1,0,3
3,1,0,1,703,0,1
4,260,2,2,1,783,42
5,335,5,88,52,26,2041
